In [3]:
from typing import TypedDict, NotRequired
from langgraph.graph import StateGraph, START, END

In [4]:
class AgentState(TypedDict):
    number1: int
    operation: str
    number2: int
    finalNumber: NotRequired[int]

In [5]:
def adder(state: AgentState) -> AgentState:
    """ 
    This node adds the 2 numbers
    """
    
    return {**state, "finalNumber": state["number1"] + state["number2"]}
    

In [6]:
def subtracter(state: AgentState) -> AgentState:
    """ 
    This node subtracts two numbers
    """
    return {**state, 'finalNumber': state['number1'] - state['number2']}

In [7]:
def decide_next_node(state: AgentState) -> str:
    """ 
    This node will decide the next node of the graph based on the operation.
    
    Args:
        state (AgentState): The current state containing the operation to perform.
        
    Returns:
        str: The name of the edge to take
        
    Raises:
        ValueError: If the operation is not supported (not '+' or '-').
    """
    
    # Check if the operation is addition.
    if state["operation"] == "+":
        return "add_operation"
    # Check if the operation is subtraction.
    elif state["operation"] == "-":
        return "subtract_operation"
    else:
        # Default case for unsupported operations.
        raise ValueError(f"Unsupported operation: {state['operation']}")

In [8]:
graph = StateGraph(AgentState)

graph.add_node("subtract_node", subtracter)
graph.add_node("add_node", adder)
graph.add_node("router", lambda state: state)  # passthrough function

graph.add_edge(START, "router")



In [9]:
# Add conditional edges from the router node.
# This creates branching logic based on the decide_next_node function's return value.
# The function will return either "add_operation" or "subtract_operation",
# which will route to "add_node" or "subtract_node" respectively.
graph.add_conditional_edges(
    "router",                    # Source node that the conditional edges start from.
    decide_next_node,           # Function that determines which edge to take.
    {
        "add_operation": "add_node",        # If function returns "add_operation", go to "add_node".
        "subtract_operation": "subtract_node"  # If function returns "subtract_operation", go to "subtract_node".
    }
)

In [10]:
graph.add_edge("add_node", END)
graph.add_edge("subtract_node", END)


In [12]:
app = graph.compile()

result = app.invoke({'number1': 2, 'number2': 3, 'operation':"+"})  

print(result)

{'number1': 2, 'operation': '+', 'number2': 3, 'finalNumber': 5}
